In [1]:
import pandas as pd
import numpy as np
import re 
import requests 
import matplotlib 
import seaborn as sns
from bs4 import BeautifulSoup 
from pandasql import sqldf
%matplotlib inline
import sqlite3

In [2]:
# FUNCTIONS CREATED 

# FUNCTION TO GET COLUMN NAMES 
def getcol(dfname):
    print(str(dfname.columns.values.tolist()))

In [3]:
#CSV files, named dataframe, type & corresponding index to help differentiate without making long unique names
# file names optimized for shift tabing  
dfcsv_1 = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
dfcsv_2 = pd.read_csv('zippedData/imdb.name.basics.csv.gz')
dfcsv_3 = pd.read_csv('zippedData/imdb.title.akas.csv.gz')
dfcsv_4 = pd.read_csv('zippedData/imdb.title.basics.csv.gz')
dfcsv_5 = pd.read_csv('zippedData/imdb.title.crew.csv.gz')
dfcsv_6 = pd.read_csv('zippedData/imdb.title.principals.csv.gz')
dfcsv_7 = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')
dfcsv_8 = pd.read_csv('zippedData/tmdb.movies.csv.gz')

In [4]:
#TSV FILES, named dataframe, file type & corresponding index to help differentiate without making long unique names
dftsv_1= pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter='\t',encoding='utf-8')
dftsv_2 = pd.read_csv('zippedData/rt.reviews.tsv.gz', delimiter='\t',encoding='unicode_escape')

In [5]:
#WEBSCRAPED DATA, named dataframe, file type & corresponding index to help differentiate without making long unique names
dfxls_1 = pd.read_excel('zippedData/df_IMDB_Akas_english.xlsx')
dfxls_2 = pd.read_excel('zippedData/df_MasterFinancials.xlsx')
dfxls_3 = pd.read_excel('zippedData/Golden_Globes.xlsx')

In [6]:
#KAGGLE DATA, named dataframe, where collected, file type & corresponding index to help differentiate without making long names
# low_memory = False reccomended by error message, still not totally sure what it does 
dfkagcsv_1 = pd.read_csv('zippedData/credits.csv', low_memory=False )
dfkagcsv_2 = pd.read_csv('zippedData/keywords.csv', low_memory=False)
dfkagcsv_3 = pd.read_csv('zippedData/links.csv', low_memory=False)
dfkagcsv_4 = pd.read_csv('zippedData/links_small.csv', low_memory=False)
dfkagcsv_5 = pd.read_csv('zippedData/movies_metadata.csv', low_memory=False)
dfkagcsv_6 = pd.read_csv('zippedData/ratings.csv', low_memory=False)
dfkagcsv_7 = pd.read_csv('zippedData/ratings_small.csv', low_memory=False)

In [7]:
#RUSS MASTER DATA, named dataframe, master, file type & corresponding index to help differentiate without making long names
dfmxls_1 = pd.read_excel('zippedData/df_Generes_With_tconst.xlsx')
dfmxls_2 = pd.read_excel('zippedData/df_IMDB_MovieCatalog.xlsx')
dfmxls_3 = pd.read_excel('zippedData/df_Movie_Financials.xlsx')

In [8]:
checking_titles = clean1.merge(clean3, how='inner', on='title', indicator='title2').merge(clean8, how='inner', on='title', indicator='title3')
# USING INDICATOR TO SEE IF VALUES MATCH ON BOTH SIDES, GOT ONE RESULT 'BOTH' FOR BOTH
#checking_titles['title3'].describe()

#checking_titles['title2'].describe()

#ALL 8 STARTING CSV 'TITLE' COLUMNS MATCH UP 

NameError: name 'clean1' is not defined

In [ ]:
#    CLEANED COLUMNS FROM ORIGINAL 10 FILES PROVIDED, ALL OF THE FLATIRON MATERIALS 
#clean1 'title', 'studio', 'foreign_gross', 'year'     INCONSISTENT DENOMINATIONS FOR FOREIGN GROSS, USE WEBSCRAPED DATA 
#clean2 'nconst', 'primary_name', 'primary_profession', 'known_for_titles'
#clean3 'title_id', 'ordering', 'title', 'region', 'is_original_title'
#clean4 'tconst', 'primary_title', 'original_title', 'start_year', 'runtime_minutes', 'genres'
#clean5 'tconst', 'directors', 'writers'
#clean6 'tconst', 'ordering', 'nconst', 'category'
#clean7 'tconst', 'averagerating', 'numvotes'
#clean8 'genre_ids', 'id', 'original_language', 'original_title', 'release_date', 'title', 'vote_average', 'vote_count'

#   THESE FOLLOWING 'ID' DO NOT CORRESPOND TO ANY OTHER ID EXCEPT EACHOTHER 
#clean9 'id', 'synopsis', 'genre', 'director', 'writer', 'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime', 'studio'
#clean10 'id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher', 'date'

In [ ]:
#    STARTING COLUMNS 
#dfcsv_1 'title', 'studio', 'domestic_gross', 'foreign_gross', 'year'
#dfcsv_2 'nconst', 'primary_name', 'birth_year', 'death_year', 'primary_profession', 'known_for_titles'
#dfcsv_3 'title_id', 'ordering', 'title', 'region', 'language', 'types', 'attributes', 'is_original_title'
#dfcsv_4 'tconst', 'primary_title', 'original_title', 'start_year', 'runtime_minutes', 'genres'
#dfcsv_5 'tconst', 'directors', 'writers'
#dfcsv_6 'tconst', 'ordering', 'nconst', 'category', 'job', 'characters'
#dfcsv_7 'tconst', 'averagerating', 'numvotes'
#dfcsv_8 'Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title', 'popularity', 'release_date', 
#        'title', 'vote_average', 'vote_count'

In [ ]:
# XLS COLUMNS FIRST 2 ARE FROM RUSSELL, THIRD MADE BY FENNEC 
#dfxls_1  'Unnamed: 0', 'tconst', 'primaryTitle', 'startYear', 'runtimeMinutes',
#         'genres', 'startYear_str', 'titleyear', 'titleId', 'ind_Link'
#dfxls_2 'Unnamed: 0', 'Unnamed: 0_IM',  'tconst', 'Domestic', 'International', 'Worldwide', 'Domestic Opening', 'Budget',
#        'Earliest Release Date', 'MPAA', 'Running Time', 'Domestic Distributor', 'Genres', 'primaryTitle_IM', 'originalTitle',
#        'isAdult', 'startYear_IM',  'runtimeMinutes_IM', 'genres_IM', 'titleyear_IM', 'Unnamed: 0_fl', 'primaryTitle_fl',
#        'startYear_fl', 'runtimeMinutes_fl', 'genres_fl', 'startYear_str', 'titleyear_fl', 'titleId', 'ind_Link', 'TN_movie',
#        'TN_year', 'TN_release_date', 'TN_production_budget', 'TN_domestic_gross', 'TN_worldwide_gross', 'TN_foreign_gross_calc',
#        'title', 'studio', 'domestic_gross', 'foreign_gross', 'year', 'worldwide_gross_calc', 'indicator_column',
#        'MergingFinances',  'mergingfinancials'

#dfxls_3 'Year', 'Drama', 'Musical/Comedy', 'Drama Actor', 'Drama Actor Movie ', 'Musical/Comedy Actor', 
#        'Musical/Comedy Actor Movie ', 'Drama Actress', 'Drama Actress Movie ', 'Musical/Comedy Actress',
#        'Musical/Comedy Actress Movie',  'Director', 'Director Movie '

In [ ]:
#   KAGGLE COLUMNS 
# kagglecsv1 'cast', 'crew', 'id'
# kagglecsv2 'id', 'keywords'
# kagglecsv3 'movieId', 'imdbId', 'tmdbId'
# kagglecsv4 'movieId', 'imdbId', 'tmdbId'
# kagglecsv5 'adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 
#            'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 
#            'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 
#            'vote_count'
# kagglecsv6 'userId', 'movieId', 'rating', 'timestamp'
# kagglecsv7'userId', 'movieId', 'rating', 'timestamp'

In [ ]:
#    TSV COLUMNS FROM FLATIRON 
#dftsv_1 'id', 'synopsis', 'rating', 'genre', 'director', 'writer', 'theater_date', 'dvd_date', 'currency', 
#        'box_office', 'runtime', 'studio'
#dftsv_2 'id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher', 'date'

In [ ]:
#    MASTER COLUMNS 
# dfmxls_1 'Unnamed: 0', 'tconst', 'titleType', 'primaryTitle', 'originalTitle', 'startYear', 'runtimeMinutes',
#           'genres', 'averageRating', 'numVotes', 'Splitgeneres'
# dfmxls_2 'Unnamed: 0', 'tconst', 'primaryTitle', 'startYear', 'runtimeMinutes', 'genres', 'startYear_str', 
#          'titleyear', 'studio_BOM', 'MPAA', 'rd', 'StudioDesc'
# dfmxls_3 'Unnamed: 0', 'tconst', 'P_Cost', 'Dom_Gross', 'Frgn_Gross', 'WW_Gross', 'Domestic Opening'

#studio_BOM seems usless, appears to be a bool value for a question not asked
# StudioDesc is studio names

In [ ]:
# NEW DATAFRAMES CREATED 
# Actorframe, data on actors from rotten tomatos, titles duplicate when they have multiple actors 
# Studioframe, data we have on studio production associated with movies, 
# data_on_numbers, no title duplicates newsest data in the flatiron provided data sorted by domestic gross

In [ ]:
dftsv_1['rating'].rename('parent_rating', inplace=True) #name slightly misleading 

#keeping non-numeric columns with few NaN values so we could still use 
#to groupby or search from, can always drop later subbing for unknown 
#so we don't get and NaN errors 

clean1 = dfcsv_1.fillna(dfcsv_1['domestic_gross'].mean())
clean2 = dfcsv_2.drop(labels='birth_year', axis=1).drop(labels='death_year', axis=1).fillna('unknown')
clean3 = dfcsv_3.drop(labels='attributes', axis=1).drop(labels='language', axis=1).drop(labels='types', axis=1).fillna('unknown')
clean4 = dfcsv_4.fillna(dfcsv_4['runtime_minutes'].median()).fillna('unknown')
clean5 = dfcsv_5.fillna('unknown')
clean6 = dfcsv_6.drop(labels='job', axis=1).drop(labels='characters', axis=1)
clean7 = dfcsv_7
#numvotes range wildly making votes less accurate if not sorted to be in certain thresholds 
clean8 = dfcsv_8.drop(labels='Unnamed: 0', axis=1).drop(labels='popularity', axis=1)
#columns full but useless 
clean9 = dftsv_2.drop(labels='rating', axis=1).fillna('unknown')
#rating probably not worth trying to add, would have to convert a lot to determine mean/med/mode
#review subbed for unknown so we could still use to search 
clean10 = dftsv_1 #Really short but lots of data, join on left & right in sep DF w/clean1 

In [ ]:
c1 = clean1.groupby(['studio', 'title']).mean()
c3 = clean3.groupby(['title_id', 'title', 'region']).mean()
#some meaningful groupings
clean1['studio']

In [ ]:
the_df_to_merge_them_all['studio'].describe()

In [ ]:
#playing around to find the largest dataframe that could help connect everything, think sql is going to be a better move 
the_df_to_merge_them_all = clean1.merge(clean3, how='outer', on='title').merge(clean8,  how='outer', on='title').merge(clean4,  how='outer', on='original_title').merge(clean5, how='outer', on='tconst').merge(clean6, how='outer', on='tconst').merge(clean7, how='outer', on='tconst').merge(clean2, how='outer', on='nconst').merge(clean7, how='outer', on='tconst')
the_df_to_merge_them_all.sort_values(by='title_id', ascending=False)
mini_all = the_df_to_merge_them_all[['title', 'studio', 'year', 'original_language','original_title', 'release_date', 'vote_average', 'vote_count', 'runtime_minutes', 'genres', 'directors', 'writers', 'id', 'tconst']].copy()

getcol(mini_all)

In [ ]:
#dfxls_2(studio) tconst  
#clean 1 (studio) title, but on a frame we know it matches with 
#clean8 (title), 

#clean1 via title > clean 8 via 
#mini_all.join(dfxls_2, on='tconst')

findin_studios = mini_all.merge(dfxls_2, on='tconst', how='outer')[['studio_y', 'studio_x', 'original_title', 'tconst']].copy()
Studioframe = findin_studios.drop_duplicates(subset=['tconst']).merge(dfmxls_2, on='tconst', how='outer')[['studio_y', 'studio_x', 'StudioDesc' , 'original_title', 'genres', 'runtimeMinutes','tconst']].copy()
Studioframe['Studio'] = Studioframe['StudioDesc'].str.cat(Studioframe['studio_x'] ,sep=", ")
Studioframe['Studio'] = Studioframe['Studio'].str.cat(Studioframe['studio_y'] ,sep=", ")

In [ ]:
Juststudios = Studioframe.dropna(axis=0, subset=['original_title', 'Studio']).copy()
Juststudios

In [ ]:
grouped_data = the_df_to_merge_them_all.groupby(['studio','original_title','domestic_gross', 'year', 'title_id', 'id', 'directors'])
#grouping some sets of data that make sense together 
grouped_data.head(5)

In [ ]:
#    DATA ON ACTORS FROM ROTTEN TOMATOES

actors = clean8.merge(clean1, how='inner', on='title').merge(clean4, how='inner', on='original_title').merge(clean6, how='inner', on='tconst').merge(clean2, how='inner', on='nconst')
actors.insert(1, 'actors_name', actors.pop('primary_name'))
actors.groupby(['actors_name', 'vote_average']).head()
Actorframe = actors[['actors_name', 'primary_title', 'release_date','vote_average', 'vote_count', 'category', 'genres' ]].copy()
Actorframe.sort_values(by='vote_count', ascending=False).head(50)
Actorframe.loc[actors['primary_title'] == 'Inception']


In [ ]:
#    NEWEST DATA BASED ON HIGHEST DOMESTIC GROSS SORTED BY TITLE 

cleaning_combining_df = the_df_to_merge_them_all.sort_values(by='tconst').drop_duplicates(subset='tconst').sort_values(by='domestic_gross', ascending=False).drop_duplicates(subset='domestic_gross', keep='first')
cleaning_combining_df_1 = cleaning_combining_df.drop(labels='region', axis=1).drop(labels='title_id', axis=1).drop(labels='directors', axis=1).drop(labels='writers', axis=1)
#duplicates dropped by most recent domestic gross 
cleaning_combining_df_2 = cleaning_combining_df_1.merge(dfmxls_2, on='id')
#I'd do them both in one line if they didn't already look monsterous 
without_useless = cleaning_combining_df_1.drop(labels='ordering_x', axis=1).drop(labels='id', axis=1).drop(labels='ordering_y', axis=1).drop(labels='averagerating_x', axis=1).drop(labels='numvotes_x', axis=1).sort_values(by='title')
without_useless.insert(0, 'original_title', without_useless.pop('original_title'))
data_on_numbers = without_useless.sort_values(by='domestic_gross', ascending=False)
FINumbersframe = data_on_numbers[[ 'original_language', 'tconst', 'nconst', 'original_title', 'primary_title', 'studio', 'runtime_minutes', 'year', 'vote_average', 'vote_count',  'genres']].copy()
#TOP 10 MOVIES WE'VE GOT PLENTY OF DATA ON 
FINumbersframe.sort_values(by='vote_average', ascending=False).dropna()


In [ ]:
data_on_numbers.loc[data_on_numbers['year'] == 2018]
#MOVIES IN 2018 SORTED BY NUMBER
the_df_to_merge_them_all['runtime_minutes'].mean()
#THE BEST RUNTIME BETWEEN GENRES

#MAYBE TRY AND FIND OUT BEST RUNTIME FOR A SPECIFIC GENRES?

In [ ]:
#     NOTE: THIS DATAFRAME HAS ALREADY BEEN SORTED 
dict_min_year = {2018: data_on_numbers.loc[data_on_numbers['year'] == 2018]['runtime_minutes'].head(10).mean(),
                 2017: data_on_numbers.loc[data_on_numbers['year'] == 2017]['runtime_minutes'].head(10).mean(),
                 2016: data_on_numbers.loc[data_on_numbers['year'] == 2016]['runtime_minutes'].head(10).mean(),
                 2015: data_on_numbers.loc[data_on_numbers['year'] == 2015]['runtime_minutes'].head(10).mean(),
                 2014: data_on_numbers.loc[data_on_numbers['year'] == 2014]['runtime_minutes'].head(10).mean(),
                 2013: data_on_numbers.loc[data_on_numbers['year'] == 2013]['runtime_minutes'].head(10).mean(),
                 2012: data_on_numbers.loc[data_on_numbers['year'] == 2012]['runtime_minutes'].head(10).mean(),
                 2011: data_on_numbers.loc[data_on_numbers['year'] == 2011]['runtime_minutes'].head(10).mean(),
                 2010: data_on_numbers.loc[data_on_numbers['year'] == 2010]['runtime_minutes'].head(10).mean()}

to_plot = pd.DataFrame.from_dict(dict_min_year, orient='index')
to_plot.plot(title='How runtimes for the top 10 films has changed in the last 8 years ')

In [ ]:
#FIXING RUNTIME COLUMNS 

dfxls_2['Running Time'].fillna('0 hr 0 min', inplace=True)

for i in range(len(dfxls_2['Running Time'])):
    newtime = 0 
    oldtime = dfxls_2['Running Time'][i].split()
    for k in range(0, len(oldtime), 2):
        if oldtime[k+1] == 'hr':
            newtime += int(oldtime[k])*60
        else: 
            newtime += int(oldtime[k])
    dfxls_2['Running Time'][i] = newtime

In [ ]:
dfxls_2.loc[dfxls_2['Running Time'] == 288]
# LONGEST MOVIE

In [ ]:
# TRYING TO MAKE A FUNCTION TO FIX HOUR/MIN COLUMNS 

#def stringtimetoint(dataframe_and_column):
    # column must contain variables in 'int', 'hour', 'int', 'min' format
 #   dataframe_and_column.fillna('0 hr 0 min', inplace=True)
    
  #  for i in range(len(dataframe_and_column)):
   #     newtime = 0 
   #     oldtime = dataframe_and_column[i].split()
   #     for k in range(0, len(oldtime), 2):
   #         if oldtime[k+1] == 'hr':
   #             newtime += int(oldtime[k])*60
   #         else: 
   #             newtime += int(oldtime[k])
   #         dataframe_and_column[i] = newtime
    
#stringtimetoint(dfxls_2['Running Time'])

questions:
best studio & director for a horror or action movie in october?

do ratings correspond in any way to box office sucess? if so, which top 5 should we s should you send your film to (critic with highest reviws highest num votes?)

sucess of video game/comic book movies in recent years, microsoft now owns fallout (could be comperable to mad max) and skyrim? we can almost certainly find data to support a skyrim movie

budget relation to box office gross? what variables might change that & what might be best?



In [ ]:
dfxls_3['Drama Actress'].str.contains('Meryl Streep').value_counts()

In [ ]:
rotten = clean9.merge(clean10, how='inner', on='id')
rotten.groupby(rotten['synopsis']).mean()